# Assignment 2: Language Modelling in Hangman

Student Name: Jiexin Liu

Student ID: 1044804

## General info

<b>Due date</b>: Tuesday, 6 April 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [ ]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [ ]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [ ]:
interactive = True

<b>For your testing:</b>

You can play the game interactively using the following command:

In [ ]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
from nltk.corpus import brown
import numpy as np
import re
import string

letter_lowercase = list(string.ascii_lowercase)

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###

words = list(set([word.lower() for word in brown.words() if not re.search(r'[^a-zA-Z]', word)]))

np.random.shuffle(words)

test_set += words[0:1000]
training_set += words[1000:]



###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

<b>For your testing:</b>

In [ ]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [ ]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [ ]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
        
    return np.random.choice([char for char in letter_lowercase if char not in list(guessed)])
    

    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

<b>For your testing:</b>

In [ ]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
unigram_counts = None

###
# Your answer BEGINS HERE
###

training_set_length = len(training_set)

unigram_counts = {}

for letter in letter_lowercase:
    unigram_counts.update({letter: 0})

for word in training_set:

    chars = list(set(word))

    for char in chars:
        unigram_counts[char] += 1


###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    
    sorted_char = [item[0] for item in sorted(unigram_counts.items(), key=lambda item: item[1], reverse=True)]

    for char in sorted_char:
        if char not in guessed:
            return char

    ###
    # Your answer ENDS HERE
    ###


result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

<b>For your testing:</b>

In [ ]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [ ]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###

# {
#   '1': {'a': 1, 'b': 2, ..., 'z': 10},
#   '2': {'a': 5, 'b': 2, ..., 'z': 10},
#   ...
#   '10': {'a': 1, 'b': 2, ..., 'z': 10}
#   ...
#
#}
# key is the word length
unigram_counts_by_length = {}

for word in training_set:

    chars = list(set(word))
    word_length = len(word)
    letter_dict = {}

    for letter in letter_lowercase:
        letter_dict.update({letter: 0})

    if str(word_length) not in unigram_counts_by_length.keys():
        unigram_counts_by_length.update({str(word_length): letter_dict})

    for char in chars:
        unigram_counts_by_length[str(word_length)][char] += 1

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    
    sorted_char_global = [item[0] for item in sorted(unigram_counts.items(), key=lambda item: item[1], reverse=True)]

    guess_word_length = len(mask)

    if str(guess_word_length) in unigram_counts_by_length.keys():
        for char in [pair[0] for pair in
                     sorted(unigram_counts_by_length.get(str(guess_word_length)).items(), key=lambda item: item[1],
                            reverse=True)]:
            if char not in guessed:
                return char
    else:
        for char in sorted_char_global:
            if char not in guessed:
                return char
    
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

<b>For your testing:</b>

In [ ]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [ ]:
bigram_counts = {}

###
# Your answer BEGINS HERE
###
letter_dict = {}  # Keep total counts for letters
for letter in letter_lowercase:
    letter_dict.update({letter: 0})

for letter in letter_lowercase:
    bigram_counts.update({letter: letter_dict.copy()})

bigram_counts.update({'$': letter_dict.copy()})

# Train bigrams model and store the total counts in each category, like this:
# {
#   'a': {'a': 1, 'b': 2, ..., 'z': 10},
#   'b': {'a': 5, 'b': 2, ..., 'z': 10},
#   ...
#   '$': {'a': 1, 'b': 2, ..., 'z': 10}
#
#}
# So 'ba' exists 5 times in this case
#
for word in training_set:

    word_list = list(word)
    word_list.insert(0, '$')

    for index, char in enumerate(word_list):

        if index == len(word_list) - 1:
            break
        else:
            bigram_counts[char][word_list[index + 1]] += 1
###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigram_counts_by_length=unigram_counts_by_length): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    word_length = len(mask)
    mask_list = mask.copy()
    mask_list.insert(0, '$')
    letter_counts = letter_dict.copy()
    flag = False

    for index in range(len(mask_list)):

        if index == len(mask_list) - 1:
            break
        elif mask_list[index] == '_' and mask_list[index + 1] == '_': # no context
            flag = True
        elif mask_list[index] != '_' and mask_list[index + 1] == '_':
            for char in letter_counts:
                letter_counts[char] += bigram_counts.get(mask_list[index]).get(char)
        else:
            continue

    if flag:  #  if no context, use unigrams model
        for char in letter_counts:
            letter_counts[char] += unigram_counts_by_length.get(str(word_length)).get(char)

    sorted_letter_counts = [char_guess[0] for char_guess in
                            sorted(letter_counts.items(), key=lambda item: item[1], reverse=True)
                            if char_guess[0] not in guessed]

    return sorted_letter_counts[0]
    ###
    # Your answer ENDS HERE
    ###



result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 8.0
* 1.0 mark if 8.0 <= x < 8.5
* 0.5 mark if 8.5 <= x < 8.8
* 0.0 mark if x >= 8.8

Note: when testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [ ]:
###
# Your answer BEGINS HERE
###
###
# Your answer BEGINS HERE
###
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.lm import MLE
from nltk.probability import FreqDist
from functools import partial
from nltk.lm.preprocessing import flatten
from nltk import everygrams
from itertools import count


def create_everygrams_reverse(order, text):
    padding_fn = partial(pad_both_ends, n=order)

    return (
        (everygrams(list(reversed(list(padding_fn(sent)))), max_len=order) for sent in text),
        flatten(map(padding_fn, text)),
    )

# The 'everygrams' is a mix of ngrams. If n is 2, then everygrams will include unigrams and bigrams.
# The 'create_everygrams_reverse' function is to create everygram based on reversed padding word. For example, the order of 
# '<s>abc</s>' is reversed like '</s>cba<s>' and then fit it into everygrams model in order to get the right-side context
# using the similar code.

def my_amazing_ai_guesser(mask, guessed):
    
    
#     # Monitor the process of testset being guessing
#     if not guessed:
#         output = next(counter)
#         print('Word Number')
#         print(output)

    letter_prob = FreqDist() # In each guess round, it keeps the probability distribution among letters from 'a' to 'z'

    chars = ['<s>', '<s>'] + mask + ['</s>', '</s>']  # Padding each word with start and end symbol. The highest order is 3.
    noContext = False

    for index in range(2, len(chars) - 2):

        unigram_flag_left = False
        unigram_flag_right = False
        
        # No need to guess if the position is filled.
        if chars[index] != '_':
            continue

        # left context calculation

        if len(model_left_context.counts[[chars[index - 2], chars[index - 1]]]):  # trigram model

            for letter in letter_lowercase:
                letter_prob[letter] += model_left_context.score(letter, [chars[index - 2], chars[index - 1]])  

        elif len(model_left_context.counts[[chars[index - 1]]]):  # bigram model

            for letter in letter_lowercase:
                letter_prob[letter] += model_left_context.score(letter, [chars[index - 1]])

        else:  # unigram model

            unigram_flag_left = True

        # right context calculation

        if len(model_right_context.counts[[chars[index + 2], chars[index + 1]]]):  # trigram model

            for letter in letter_lowercase:
                letter_prob[letter] += model_right_context.score(letter, [chars[index + 2], chars[index + 1]])

        elif len(model_right_context.counts[[chars[index + 1]]]):  # bigram model

            for letter in letter_lowercase:
                letter_prob[letter] += model_right_context.score(letter, [chars[index + 1]])

        else:  # unigram model

            unigram_flag_right = True

        # if there is no context from left and right side, use unigram model
        if unigram_flag_left and unigram_flag_right:
            noContext = True

    if noContext:
        for letter in letter_lowercase:
            letter_prob[letter] += model_left_context.score(letter)

    guess_sorted = [pair[0] for pair in sorted(letter_prob.items(), key=lambda item: item[1], reverse=True)
                    if pair[0] not in guessed]

    return guess_sorted[0]
        
    
def test_different_split(num):
    
    print('Seed Number: ', num)
    
    global test_set
    global training_set
    
    np.random.seed(num)

    words = list(set([word.lower() for word in brown.words() if not re.search(r'[^a-zA-Z]', word)]))

    np.random.shuffle(words)

    test_set += words[0:1000]
    training_set += words[1000:]
    
    model_left_context = MLE(3)
    model_right_context = MLE(3)
    train_left, vocab_left = padded_everygram_pipeline(3, training_set)  # highest order is 3, which means trigrams are involved.
    train_right, vocal_right = create_everygrams_reverse(3, training_set)
    model_left_context.fit(train_left, vocab_left)  # ngrams model involves left-side context
    model_right_context.fit(train_right, vocal_right) 
    
    result = test_guesser(my_amazing_ai_guesser)
    print("Average number of incorrect guesses: ", result)
    return result

print('Building models...')
model_left_context = MLE(3)
model_right_context = MLE(3)
train_left, vocab_left = padded_everygram_pipeline(3, training_set)  # highest order is 3, which means trigrams are involved.
train_right, vocal_right = create_everygrams_reverse(3, training_set)
print('Train model 1 out of 2...')
model_left_context.fit(train_left, vocab_left)  # ngrams model involves left-side context
print('Left-side context model is set up.')
print('Train model 2 out of 2...')
model_right_context.fit(train_right, vocal_right)  # ngrams model involves right-side context
print('Right-side context model is set up.')


counter = count()  # 

# # This is a test among different dataset with different seeds, which runs longer time...
# result = 0
# seed_num = 5
# for i in range(seed_num):
    
#     print('Start testing different dataset...')
#     result += test_different_split(i)
#     print('Finish No.', i, ' test...')
#     print()

# print('Average number of incorrect guesses among different dataset: ', float(result) / seed_num)
###
# Your answer ENDS HERE
###
print('Start guessing...')
result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE



##### Your answer ENDS HERE

In Qustion 6, I try to use nltk package to build language model rathner than manually code. The basic language model is called 'everygrams', which can be found in this link: https://www.nltk.org/api/nltk.html?highlight=everygrams#nltk.util.everygrams. In Question 6, I specify the highest order of everygrams model to be 3. That means this ngrams model can involve unigrams, bigrams, and trigrams together in one model. Also, bidirectional model is applied. The algorithm first iterates the mask list to find '_' and check its context. Left side context is explored first. Then using trained model to get conditional probability based on the trigrams model. If it cannot be found, then goes to bigrams model. The algorithm also sum up all probability for letters 'a' to 'z' and stores them in 'letter_prob'. When left side context exploring is finished, the algorithm starts checking right side context based on the similar workflow. The loop will continue until all the unfilled letters' context are explored. Finally, the most frequent letter is chosen.